# Exploration 4

### 멋진 작사가 만들기 🎵

### Table of Contents

1. [데이터 로드]()
2. [데이터 전처리]()
   - [정규표현식]()
   - [토큰화]()
3. [훈련, 테스트셋 분리]()
4. [모델 설계 및 훈련]()
5. [모델 평가]()
6. [회고]()


## 1. 데이터 로드

In [1]:
import re
import numpy as np
import tensorflow as tf
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:20])

데이터 크기: 187088
Examples:
 ['[Hook]', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face', "I've been down so long, it look like up to me", 'They look up to me', "I got fake people showin' fake love to me", 'Straight up to my face, straight up to my face [Verse 1]', "Somethin' ain't right when we talkin'", "Somethin' ain't right when we talkin'", "Look like you hidin' your problems", 'Really you never was solid', 'No, you can\'t "son" me', "You won't never get to run me", 'Just when shit look out of reach', 'I reach back like one, three', 'Like one, three, yeah [Pre-Hook]', "That's when they smile in my face", 'Whole time they wanna take my place']


## 2. 데이터 전처리
### 2-1. 정규표현식

In [2]:
# 정규표현식으로 문장 정제하는 함수
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀝니다.
    sentence = re.sub(r'\[[^)]*\]', '', sentence)         # [Hook]처럼 대괄호로 파트 구분하는 문자 제거
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)    # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)           # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 문장 앞뒤에 <start>와 <end> 추가
    
    return sentence

print(preprocess_sentence(raw_corpus[0]))   # 문장이 어떻게 필터링되는지 확인

<start>  <end>


In [3]:
# 정제된 corpus 생성
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0 or len(sentence.split()) > 12: continue   # 길이가 0이거나 단어가 12개 넘는 긴 문장은 제외
    if preprocess_sentence(sentence) == '<start>  <end>': continue  # 공백만 있는 문장 제외
    corpus.append(preprocess_sentence(sentence))

print(len(corpus))
corpus[:20]

158297


['<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> i ve been down so long , it look like up to me <end>',
 '<start> they look up to me <end>',
 '<start> i got fake people showin fake love to me <end>',
 '<start> straight up to my face , straight up to my face <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> somethin ain t right when we talkin <end>',
 '<start> look like you hidin your problems <end>',
 '<start> really you never was solid <end>',
 '<start> no , you can t son me <end>',
 '<start> you won t never get to run me <end>',
 '<start> just when shit look out of reach <end>',
 '<start> i reach back like one , three <end>',
 '<start> like one , three , yeah <end>',
 '<start> that s when they smile in my face <end>',
 '<start> whole time they wanna take my place <end>',
 '<sta

### 2-2. 토큰화

In [4]:
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,   # 전체 단어의 개수 
        filters=' ',       # 별도로 전처리 로직을 추가할 수 있습니다. 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 됩니다.

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축하게 됩니다.
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환합니다.

    # padding으로 입력 데이터의 시퀀스 길이를 일정하게 맞추기 (maxlen 15로 설정) 
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post', maxlen=15)  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

In [5]:
# 변환된 텐서 확인
print(tensor[:3])
tensor.shape

[[   2    5   91  103   59   31  166    4   11  133   24   29   10   12
     3]
 [   2   42  133   29   10   12    3    0    0    0    0    0    0    0
     0]
 [   2    5   40  828  172 2394  828   37   10   12    3    0    0    0
     0]]


(158297, 15)

In [6]:
# 단어사전 확인
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [7]:
# source, target 문장 생성
src_input = tensor[:, :-1]   # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.

print(src_input[0])
print(tgt_input[0])
src_input.shape

[  2   5  91 103  59  31 166   4  11 133  24  29  10  12]
[  5  91 103  59  31 166   4  11 133  24  29  10  12   3]


(158297, 14)

## 3. 훈련, 테스트셋 분리

In [8]:
from sklearn.model_selection import train_test_split

# tensor를 train, test 데이터로 분리
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2)

print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (126637, 14)
Target Train: (126637, 14)


## 4. 모델 설계 및 훈련

In [9]:
vocab_size = tokenizer.num_words + 1  # 단어사전의 단어 개수 + 0:<pad>
embedding_size = 256
hidden_size = 1024

model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocab_size, embedding_size))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.LSTM(hidden_size, return_sequences=True))
model.add(tf.keras.layers.Dense(vocab_size))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 256)         3072256   
_________________________________________________________________
lstm (LSTM)                  (None, None, 1024)        5246976   
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 1024)        8392704   
_________________________________________________________________
dense (Dense)                (None, None, 12001)       12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________


In [10]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10)

Epoch 1/10
3958/3958 [==============================] - 140s 35ms/step - loss: 3.1474
Epoch 2/10
3958/3958 [==============================] - 142s 36ms/step - loss: 2.7386
Epoch 3/10
3958/3958 [==============================] - 140s 35ms/step - loss: 2.4893
Epoch 4/10
3958/3958 [==============================] - 141s 36ms/step - loss: 2.2553
Epoch 5/10
3958/3958 [==============================] - 144s 36ms/step - loss: 2.0483
Epoch 6/10
3958/3958 [==============================] - 147s 37ms/step - loss: 1.8670
Epoch 7/10
3958/3958 [==============================] - 143s 36ms/step - loss: 1.7079
Epoch 8/10
3958/3958 [==============================] - 138s 35ms/step - loss: 1.5666
Epoch 9/10
3958/3958 [==============================] - 146s 37ms/step - loss: 1.4445
Epoch 10/10
3958/3958 [==============================] - 147s 37ms/step - loss: 1.3400


## 5. 모델 평가

In [13]:
# 문장 생성 함수
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환합니다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 합니다. 
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력합니다. 
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 우리 모델이 예측한 마지막 단어가 바로 새롭게 생성한 단어가 됩니다. 

        # 우리 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줍니다. 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 합니다.
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환합니다. 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 이것이 최종적으로 모델이 생성한 자연어 문장입니다.

In [12]:
# 문장 생성 함수 실행하여 모델에게 작문 시켜보기
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , i love you <end> '

In [16]:
generate_text(model, tokenizer, init_sentence="<start> i was", max_len=20)

'<start> i was born to make you happy <end> '

In [21]:
generate_text(model, tokenizer, init_sentence="<start> i hate", max_len=20)

'<start> i hate you because i love u <end> '

In [24]:
generate_text(model, tokenizer, init_sentence="<start> you are", max_len=20)

'<start> you are the one so i make sure i behave <end> '

In [30]:
generate_text(model, tokenizer, init_sentence="<start> happy", max_len=20)

'<start> happy birthday , happy birthday , happy birthday woo , shake ! <end> '

# 6. 회고
-  하이퍼 파라미터 튜닝을 했지만 뚜렷한 성능 향상을 보이지 못했고 하이퍼 파라미터 튜닝 방법에 대해 추가적인 공부가 필요할 것 같다.


- 자연어 처리에서 정규표현식을 잘 활용하는 것은 아주 중요하다는 것을 공부했고 tensorflow API의 Tokenizer를 사용해 어떻게 전처리해서 Corpus를 생성하는지 배울 수 있었다. 


- 루브릭 지표를 맞추기 위해 간략한 전처리, embedding size, hidden size 등의 조작을 하였으나 그 과정에서 중복된 단어/문장 제외, batch_size, embbedding size, hidden size를 어떻게 조정할지 고민해봐야 할 것 같다.